### Variational Quantum Approach for the 1D Viscous Bergers Equation
#### Hardware

In [9]:
# Imports

import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter
from qiskit.visualization import circuit_drawer, plot_histogram, plot_state_city
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.visualization import circuit_drawer
%matplotlib inline

In [ ]:

your_api_key = ""
your_crn = ""

from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token=your_api_key,
    instance=your_crn,
    name="proj",
    overwrite=True
)

In [11]:
#Parameters and initialization

# ====================== PARAMETERS ======================
nu = 0.01
L = 1.0
T_max = 1.0
n_x = 2
n_t = 2
depth = 3
Nx = 4
Nt = 4
eta1 = 50
eta2 = 50

n_qubits = 1 + n_x + n_t
total_basis = 2**(n_qubits - 1)

x_grid = np.linspace(0, L, Nx)
t_grid = np.linspace(0, T_max, Nt)
X, T = np.meshgrid(x_grid, t_grid, indexing='ij')

In [12]:
#Chebyshev Basis calculation

def compute_chebyshev(k, x, a=-1, b=1):
    x_mapped = 2*(x - a)/(b - a) - 1 if b != a else 0

    T = np.zeros(k+1)
    dT = np.zeros(k+1)
    d2T = np.zeros(k+1)

    if k >= 0:
        T[0] = 1
    if k >= 1:
        T[1] = x_mapped
        dT[1] = 1

    for i in range(2, k+1):
        T[i] = 2 * x_mapped * T[i-1] - T[i-2]
        dT[i] = 2 * T[i-1] + 2 * x_mapped * dT[i-1] - dT[i-2]
        d2T[i] = 4 * dT[i-1] + 2 * x_mapped * d2T[i-1] - d2T[i-2]

    scale = 2 / (b - a) if b != a else 1
    dT *= scale
    d2T *= scale**2

    return T, dT, d2T


In [13]:
#Precompute basis values

basis_vals = {}
for i, x in enumerate(x_grid):
    for j, t in enumerate(t_grid):
        key = (i, j)
        basis_vals[key] = {
            'T': np.zeros(total_basis),
            'dT_dx': np.zeros(total_basis),
            'dT_dt': np.zeros(total_basis),
            'd2T_dx2': np.zeros(total_basis),
        }

        Tx, dTx, d2Tx = compute_chebyshev(2**n_x - 1, x, a=0, b=L)
        Tt, dTt, d2Tt = compute_chebyshev(2**n_t - 1, t, a=0, b=T_max)

        for idx in range(total_basis):
            kx = idx // (2**n_t)
            kt = idx % (2**n_t)
            basis_vals[key]['T'][idx] = Tx[kx] * Tt[kt]
            basis_vals[key]['dT_dx'][idx] = dTx[kx] * Tt[kt]
            basis_vals[key]['dT_dt'][idx] = Tx[kx] * dTt[kt]
            basis_vals[key]['d2T_dx2'][idx] = d2Tx[kx] * Tt[kt]

In [14]:
# Quantum circuit of VQC

def vqc(theta):
    qc = QuantumCircuit(n_qubits,n_qubits)
    
    # Initial layer
    for wire in range(n_qubits):
        qc.ry(theta[wire], wire)
    
    param_idx = n_qubits
    for _ in range(depth):
        for wire in range(n_qubits - 1):
            qc.cx(wire, wire + 1)
        qc.cx(n_qubits - 1, 0)

        for wire in range(n_qubits):
            qc.ry(theta[param_idx], wire)
            param_idx += 1
    
    qc.measure_all()

    return qc

In [15]:
#State vector prediction
from qiskit_ibm_runtime import Options, SamplerV2 as Sampler
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(name="proj")
service.saved_accounts()

def get_hardware_samples(theta, backend, shots=1024):
    qc = vqc(theta)  # Use your updated vqc function
    qc = transpile(qc, backend)
    
    # Direct execution without Session (for open plan)
    sampler = Sampler(mode=backend)
    job = sampler.run([qc], shots=shots)
    result = job.result()
    counts = result[0].data.meas.get_counts()
    
    return counts

def u_pred_hardware(i, j, params, backend, shots=1024):
    theta = params[:-1]
    lam = params[-1]
    
    # Get measurement counts from hardware
    counts = get_hardware_samples(theta, backend, shots)
    
    # Convert counts to probabilities
    total_shots = sum(counts.values())
    prob = np.zeros(2**n_qubits)
    
    for bitstring, count in counts.items():
        idx = int(bitstring, 2)
        prob[idx] = count / total_shots
    
    # Calculate expectation value
    key = (i, j)
    u_val = 0
    for idx in range(total_basis):
        u_val += prob[idx] * basis_vals[key]['T'][idx]
        u_val -= prob[idx + total_basis] * basis_vals[key]['T'][idx]
    
    return lam * u_val

In [16]:
def pde_residual_hardware(i, j, params, backend, shots=1024):
    theta = params[:-1]
    lam = params[-1]
    
    # Get measurement counts from hardware
    counts = get_hardware_samples(theta, backend, shots)
    
    # Convert counts to probabilities
    total_shots = sum(counts.values())
    prob = np.zeros(2**n_qubits)
    
    for bitstring, count in counts.items():
        idx = int(bitstring, 2)
        prob[idx] = count / total_shots
    
    key = (i, j)
    
    # Calculate all derivatives using hardware probabilities
    u = lam * sum(prob[idx] * basis_vals[key]['T'][idx] -
                  prob[idx + total_basis] * basis_vals[key]['T'][idx]
                  for idx in range(total_basis))
    
    u_x = lam * sum(prob[idx] * basis_vals[key]['dT_dx'][idx] -
                    prob[idx + total_basis] * basis_vals[key]['dT_dx'][idx]
                    for idx in range(total_basis))
    
    u_t = lam * sum(prob[idx] * basis_vals[key]['dT_dt'][idx] -
                    prob[idx + total_basis] * basis_vals[key]['dT_dt'][idx]
                    for idx in range(total_basis))
    
    u_xx = lam * sum(prob[idx] * basis_vals[key]['d2T_dx2'][idx] -
                     prob[idx + total_basis] * basis_vals[key]['d2T_dx2'][idx]
                     for idx in range(total_basis))
    
    return u_t + u * u_x - nu * u_xx

In [17]:
def loss_function_hardware(params, backend, shots=1024):
    total_loss = 0.0
    
    # 1. Initial condition (t = 0)
    ic_loss = 0.0
    for i in range(Nx):
        u_pred_val = u_pred_hardware(i, 0, params, backend, shots)
        u_true = 1.0 if x_grid[i] <= 0.5 else 0.0
        ic_loss += (u_pred_val - u_true)**2
    total_loss += eta1 * ic_loss / Nx
    
    # 2. Boundary condition loss
    bc_loss = 0.0
    for j in range(Nt):
        u_pred_left = u_pred_hardware(0, j, params, backend, shots)
        u_pred_right = u_pred_hardware(Nx - 1, j, params, backend, shots)
        bc_loss += (u_pred_left - 1.0)**2
        bc_loss += (u_pred_right - 0.0)**2
    total_loss += eta2 * bc_loss / (2 * Nt)
    
    # 3. PDE residual loss
    pde_loss = 0.0
    for i in range(1, Nx - 1):
        for j in range(1, Nt):
            res = pde_residual_hardware(i, j, params, backend, shots)
            pde_loss += res**2
    total_loss += pde_loss / ((Nx - 2) * (Nt - 1))
    
    return total_loss

In [ ]:
# Optimization
num_params = n_qubits * (depth + 1) + 1  # +1 for lambda scaling
init_params = np.random.uniform(0, 2 * np.pi, num_params)

# Setup hardware backend
  # Add your credentials
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=n_qubits)

# Create wrapper function for hardware optimization
def objective_function(params):
    return loss_function_hardware(params, backend, shots=1024)

print("Optimizing...")
result = minimize(objective_function, init_params, method='COBYLA', options={'maxiter': 50})
opt_params = result.x
print(f"Optimization complete! Final loss: {result.fun:.6f}")

Optimizing...


In [ ]:
#Solution grid and visualization

U = np.zeros((Nx, Nt))
for i in range(Nx):
    for j in range(Nt):
        U[i, j] = u_pred_hardware(i, j, opt_params, backend, shots=1024)


In [ ]:
# Heatmap

plt.figure(figsize=(10, 6))
plt.pcolormesh(t_grid, x_grid, U, shading='auto', cmap='viridis')
plt.colorbar(label='Velocity u(x,t)')
plt.xlabel('Time (t)')
plt.ylabel('Space (x)')
plt.title('Burgers Equation Solution: Space-Time Evolution')
plt.savefig('burgers_2d.png')
plt.show()

In [ ]:
#3D Surface plot

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
T_mesh, X_mesh = np.meshgrid(t_grid, x_grid)
surf = ax.plot_surface(X_mesh, T_mesh, U, cmap='coolwarm', rstride=1, cstride=1, alpha=0.8)
ax.set_xlabel('Space (x)')
ax.set_ylabel('Time (t)')
ax.set_zlabel('Velocity u(x,t)')
ax.set_title('Burgers Equation Solution Surface')
fig.colorbar(surf, ax=ax, shrink=0.5)
plt.savefig('burgers_3d.png')
plt.show()

In [ ]:
# Time slice animation

fig, ax = plt.subplots(figsize=(10, 6))
line, = ax.plot([], [], 'r-', linewidth=2)
ax.set_xlim(0, L)
ax.set_ylim(-0.2, 1.2)
ax.set_xlabel('Space (x)')
ax.set_ylabel('Velocity u(x,t)')
ax.grid(True)
ax.set_title('Burgers Equation Time Evolution')

def init():
    line.set_data([], [])
    return line,

def animate(j):
    u_slice = U[:, j]
    line.set_data(x_grid, u_slice)
    ax.set_title(f'Time Evolution: t = {t_grid[j]:.2f}')
    return line,

ani = FuncAnimation(fig, animate, frames=Nt, init_func=init, blit=True, interval=200)
ani.save('burgers_evolution.gif', writer='pillow', fps=5)
print("Saved burgers_evolution.gif")

In [ ]:
#Initial and final state

plt.figure(figsize=(10, 6))
plt.plot(x_grid, U[:, 0], 'b-', label='Initial: t=0', linewidth=2)
plt.plot(x_grid, U[:, -1], 'r--', label=f'Final: t={T_max:.1f}', linewidth=2)
plt.xlabel('Space (x)')
plt.ylabel('Velocity u(x,t)')
plt.legend()
plt.grid(True)
plt.title('Initial vs Final Velocity Profile')
plt.savefig('burgers_comparison.png')
plt.show()

print("All visualizations saved!")

In [ ]:
# ====================== VALIDATION & BENCHMARK ======================

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import time

# 3. Validation & Benchmark Implementation

def classical_burgers_solver(nu, L, T_max, Nx, Nt, initial_condition):
    """
    Classical finite difference solver for 1D Burgers equation
    ∂u/∂t + u∂u/∂x = ν∂²u/∂x²
    """
    dx = L / (Nx - 1)
    dt = T_max / (Nt - 1)
    
    # Stability check
    cfl = dt / dx
    diffusion_number = nu * dt / (dx**2)
    print(f"CFL number: {cfl:.4f}, Diffusion number: {diffusion_number:.4f}")
    
    x = np.linspace(0, L, Nx)
    t = np.linspace(0, T_max, Nt)
    
    # Initialize solution array
    U_classical = np.zeros((Nx, Nt))
    
    # Initial condition
    for i in range(Nx):
        U_classical[i, 0] = initial_condition(x[i])
    
    # Boundary conditions (example: u(0,t) = 1, u(L,t) = 0)
    for j in range(Nt):
        U_classical[0, j] = 1.0
        U_classical[-1, j] = 0.0
    
    # Time stepping using explicit finite difference
    for j in range(Nt - 1):
        for i in range(1, Nx - 1):
            # Convective term (upwind scheme for stability)
            if U_classical[i, j] >= 0:
                du_dx = (U_classical[i, j] - U_classical[i-1, j]) / dx
            else:
                du_dx = (U_classical[i+1, j] - U_classical[i, j]) / dx
            
            # Diffusive term (central difference)
            d2u_dx2 = (U_classical[i+1, j] - 2*U_classical[i, j] + U_classical[i-1, j]) / (dx**2)
            
            # Time update
            U_classical[i, j+1] = (U_classical[i, j] - 
                                 dt * U_classical[i, j] * du_dx + 
                                 dt * nu * d2u_dx2)
    
    return U_classical, x, t

def compute_l2_error(U_quantum, U_classical):
    """Compute L2 error between quantum and classical solutions"""
    error = np.sqrt(np.sum((U_quantum - U_classical)**2))
    relative_error = error / np.sqrt(np.sum(U_classical**2))
    return error, relative_error

def benchmark_comparison(opt_params_quantum, opt_params_classical=None):
    """
    Perform quantitative comparison between quantum and classical solvers
    """
    print("=" * 60)
    print("VALIDATION & BENCHMARK ANALYSIS")
    print("=" * 60)
    
    # Define initial condition (shock tube)
    def initial_condition(x):
        return 1.0 if x <= 0.5 else 0.0
    
    # Classical solution
    print("Computing classical reference solution...")
    start_time = time.time()
    U_classical, x_classical, t_classical = classical_burgers_solver(
        nu, L, T_max, Nx, Nt, initial_condition
    )
    classical_time = time.time() - start_time
    
    # Quantum solution (assuming you have U from your quantum solver)
    print("Extracting quantum solution...")
    start_time = time.time()
    U_quantum = np.zeros((Nx, Nt))
    for i in range(Nx):
        for j in range(Nt):
            U_quantum[i, j] = u_pred(i, j, opt_params_quantum)  # Your quantum prediction
    quantum_time = time.time() - start_time
    
    # L2 Error Analysis
    l2_error, rel_l2_error = compute_l2_error(U_quantum, U_classical)
    
    # Time step analysis (≥3 time steps)
    time_steps = [0, Nt//3, 2*Nt//3, Nt-1]
    step_errors = []
    
    for step in time_steps:
        if step < Nt:
            step_error = np.sqrt(np.sum((U_quantum[:, step] - U_classical[:, step])**2))
            step_errors.append(step_error)
    
    # Results
    print(f"\n📊 BENCHMARK RESULTS:")
    print(f"   L2 Error (absolute): {l2_error:.6f}")
    print(f"   L2 Error (relative): {rel_l2_error:.6f}")
    print(f"   Classical wall-clock time: {classical_time:.2f} seconds")
    print(f"   Quantum wall-clock time: {quantum_time:.2f} seconds")
    print(f"   Speedup factor: {classical_time/quantum_time:.2f}x")
    
    print(f"\n📈 TIME STEP ERRORS:")
    for i, (step, error) in enumerate(zip(time_steps, step_errors)):
        t_val = t_classical[step] if step < len(t_classical) else T_max
        print(f"   t={t_val:.2f}: L2 error = {error:.6f}")
    
    return {
        'l2_error': l2_error,
        'relative_l2_error': rel_l2_error,
        'classical_time': classical_time,
        'quantum_time': quantum_time,
        'step_errors': step_errors,
        'U_classical': U_classical,
        'U_quantum': U_quantum
    }

# Noisy simulator comparison
def noisy_simulator_analysis(opt_params, backend):
    """
    Compare ideal vs noisy quantum results
    """
    print("\n🔬 NOISY SIMULATOR ANALYSIS:")
    
    # Get noise model from real backend
    noise_model = NoiseModel.from_backend(backend)
    
    # Ideal simulator
    ideal_simulator = AerSimulator(method='statevector')
    
    # Noisy simulator  
    noisy_simulator = AerSimulator(noise_model=noise_model)
    
    # Compare results at key points
    test_points = [(1, 1), (Nx//2, Nt//2), (Nx-2, Nt-1)]
    
    for i, j in test_points:
        # Ideal result
        qc = vqc(opt_params[:-1])
        qc = transpile(qc, ideal_simulator)
        ideal_result = ideal_simulator.run(qc).result()
        ideal_state = ideal_result.get_statevector(qc)
        ideal_prob = np.abs(ideal_state)**2
        
        # Noisy result (use measurements)
        qc_meas = vqc(opt_params[:-1])
        qc_meas.measure_all()
        qc_meas = transpile(qc_meas, noisy_simulator)
        noisy_result = noisy_simulator.run(qc_meas, shots=1024).result()
        noisy_counts = noisy_result.get_counts()
        
        # Convert to probabilities
        total_shots = sum(noisy_counts.values())
        noisy_prob = np.zeros(2**n_qubits)
        for bitstring, count in noisy_counts.items():
            idx = int(bitstring, 2)
            noisy_prob[idx] = count / total_shots
        
        # Compute effective error rate
        fidelity = np.abs(np.sum(np.sqrt(ideal_prob * noisy_prob)))**2
        error_rate = 1 - fidelity
        
        print(f"   Point ({i},{j}): Error rate = {error_rate:.4f}")

# 4. Resource & Noise Analysis Implementation

def resource_analysis():
    """
    Analyze quantum resource usage
    """
    print("\n⚙️  RESOURCE & NOISE ANALYSIS:")
    print("=" * 40)
    
    # Create sample circuit to analyze
    sample_params = np.random.uniform(0, 2*np.pi, n_qubits * (depth + 1))
    qc = vqc(sample_params)
    
    # Count resources
    n_qubits_used = qc.num_qubits
    n_cx_gates = qc.count_ops().get('cx', 0)
    total_cx_depth = n_cx_gates  # Approximate
    n_rotation_gates = qc.count_ops().get('ry', 0)
    
    # Gate depth analysis
    qc_transpiled = transpile(qc, optimization_level=0)
    circuit_depth = qc_transpiled.depth()
    
    # T-count (for RY gates - approximate)
    # RY gates can be decomposed into Clifford + T gates
    approx_t_count = n_rotation_gates * 3  # Rough estimate
    
    print(f"📊 QUANTUM RESOURCES:")
    print(f"   Qubits used: {n_qubits_used}")
    print(f"   Two-qubit gates (CX): {n_cx_gates}")
    print(f"   Two-qubit gate depth: {total_cx_depth}")
    print(f"   Total circuit depth: {circuit_depth}")
    print(f"   Rotation gates (RY): {n_rotation_gates}")
    print(f"   Approximate T-count: {approx_t_count}")
    
    print(f"\n🛡️  MITIGATION STRATEGIES:")
    print(f"   • Zero Noise Extrapolation (ZNE): Extrapolate to zero noise limit")
    print(f"   • Clifford Data Regression (CDR): Use Clifford circuits for error mitigation")  
    print(f"   • Readout Error Mitigation: Calibrate and correct measurement errors")
    print(f"   • Dynamical Decoupling: Insert identity gates to suppress decoherence")
    
    return {
        'qubits': n_qubits_used,
        'cx_gates': n_cx_gates,
        'circuit_depth': circuit_depth,
        'rotation_gates': n_rotation_gates,
        'approx_t_count': approx_t_count
    }

results = benchmark_comparison(opt_params)
resource_info = resource_analysis()
noisy_simulator_analysis(opt_params, backend)